# Section: Encrypted Deep Learning

- Lesson: Reviewing Additive Secret Sharing
- Lesson: Encrypted Subtraction and Public/Scalar Multiplication
- Lesson: Encrypted Computation in PySyft
- Project: Build an Encrypted Database
- Lesson: Encrypted Deep Learning in PyTorch
- Lesson: Encrypted Deep Learning in Keras
- Final Project

# Lesson: Reviewing Additive Secret Sharing

_For more great information about SMPC protocols like this one, visit https://mortendahl.github.io. With permission, Morten's work directly inspired this first teaching segment._

In [0]:
import random
import numpy as np

BASE = 10

PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8
Q = 293973345475167247070445277780365744413

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL

assert(Q > BASE**PRECISION)

def encode(rational):
    upscaled = int(rational * BASE**PRECISION_FRACTIONAL)
    field_element = upscaled % Q
    return field_element

def decode(field_element):
    upscaled = field_element if field_element <= Q/2 else field_element - Q
    rational = upscaled / BASE**PRECISION_FRACTIONAL
    return rational

def encrypt(secret):
    first  = random.randrange(Q)
    second = random.randrange(Q)
    third  = (secret - first - second) % Q
    return [first, second, third]

def decrypt(sharing):
    return sum(sharing) % Q

def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [0]:
x = encrypt(encode(5.5))
x

[34503008860121728558220447094759759050,
 110977317295877189596451182397309101322,
 148493019319168328915773648288846884041]

In [0]:
y = encrypt(encode(2.3))
y

[95518102768283605434749794987837409592,
 161161446605525569251759860964605016629,
 37293796101358072383935621828153318191]

In [0]:
z = add(x,y)
z

(175625278757526625526046707543410525919,
 21497124117666852391520974989661764542,
 96850942599973769152877595248073453951)

In [0]:
decode(decrypt(z))

7.79999999

# Lesson: Encrypted Subtraction and Public/Scalar Multiplication

In [0]:
field = 23740629843760239486723

In [0]:
x = 5

bob_x_share = 2372385723 # random number
alices_x_share = field - bob_x_share + x

In [0]:
(bob_x_share + alices_x_share) % field

5

In [0]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [0]:
((bob_x_share + alice_x_share) - (bob_y_share + alice_y_share)) % field

4

In [0]:
((bob_x_share - bob_y_share) + (alice_x_share - alice_y_share)) % field

4

In [0]:
bob_x_share + alice_x_share + bob_y_share + alice_y_share

26

In [0]:
bob_z_share = (bob_x_share - bob_y_share)
alice_z_share = (alice_x_share - alice_y_share)

In [0]:
(bob_z_share + alice_z_share) % field

4

In [0]:
def sub(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] - b[i]) % Q)
    return tuple(c)

In [0]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [0]:
bob_x_share + alice_x_share

15

In [0]:
bob_y_share + alice_y_share

11

In [0]:
((bob_y_share * 3) + (alice_y_share * 3)) % field

3

In [0]:
def imul(a, scalar):
    
    # logic here which can multiply by a public scalar
    
    c = list()
    
    for i in range(len(a)):
        c.append((a[i] * scalar) % Q)
        
    return tuple(c)

In [0]:
x = encrypt(encode(5.5))
x

[14377519562688087180292531072798533660,
 5072973488825539420145495081428746042,
 274522852423653620470007251626688464711]

In [0]:
z = imul(x, 3)

In [0]:
decode(decrypt(z))

16.5

# Lesson: Encrypted Computation in PySyft

In [0]:
!pip install syft

In [2]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0828 04:15:10.287438 140219225651072 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0828 04:15:10.306574 140219225651072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
bob = sy.VirtualWorker(hook, id="bob")#.add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice")#.add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker")#.add_worker(sy.local_worker)

In [0]:
bob._objects

{}

In [0]:
alice._objects

{}

In [0]:
secure_worker._objects

{}

In [0]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [0]:
hook.local_worker._objects

{}

In [0]:
me._objects

{}

In [0]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

In [0]:
x = x.share(bob, alice, crypto_provider=secure_worker)

In [0]:
y = y.share(bob, alice, crypto_provider=secure_worker)

In [0]:
z = x + y
z.get()

tensor([3, 1, 4, 4])

In [0]:
z = x - y
z.get()

tensor([-1,  3,  2,  4])

In [0]:
z = x * y
z.get()

tensor([ 2, -2,  3,  0])

In [0]:
z = x > y
z.get()

tensor([0, 1, 1, 1])

In [0]:
z = x < y
z.get()

tensor([1, 0, 0, 0])

In [0]:
z = x == y
z.get()

tensor([0, 0, 0, 0])

In [0]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

x = x.fix_precision().share(bob, alice, crypto_provider=secure_worker)
y = y.fix_precision().share(bob, alice, crypto_provider=secure_worker)

In [0]:
z = x + y
z.get().float_precision()

tensor([3., 1., 4., 4.])

In [0]:
z = x - y
z.get().float_precision()

tensor([-1.,  3.,  2.,  4.])

In [0]:
z = x * y
z.get().float_precision()

tensor([ 2., -2.,  3.,  0.])

In [0]:
z = x > y
z.get().float_precision()

tensor([0., 1., 1., 1.])

In [0]:
z = x < y
z.get().float_precision()

tensor([1., 0., 0., 0.])

In [0]:
z = x == y
z.get().float_precision()

tensor([0., 0., 0., 0.])

# Project: Build an Encrypted Database

In [0]:
# try this project here!
import string

In [0]:
char2int = {}
int2char = {}

In [0]:
for i,char in enumerate(' ' + string.ascii_lowercase + '0123456789' + string.punctuation):
  char2int[char] = i
  int2char [i] = char

In [0]:
char2int

In [0]:
int2char

In [0]:
max_len = 8
str_input = "Hello"

In [0]:
def string2values (str_input, max_len = 8):
  
    str_input = str_input[:max_len].lower()
    
    #pad strings shorte than max len
    if (len(str_input)<max_len):
        str_input = str_input + "."*(max_len -len(str_input))
        
    values= list()
    for char in str_input :
        values.append(char2int[char])
        
    return th.tensor(values).long()
  
  
def values2string(input_value):
    
    s = ""
    
    for value in input_value:
      s += int2char[int(value)]
    return s
        
    

In [0]:
def one_hot(index,length):
    vect =  th.zeros(length).long()
    vect[index] = 1
    return vect

In [0]:
def string2one_hot_matrix(str_input,max_len = 8):
    
    str_input = str_input[:max_len].lower()
    
    #pad strings shorte than max len
    if (len(str_input)<max_len):
        str_input = str_input + "."*(max_len -len(str_input))
    
    char_vectors = list()
    for char in str_input:
        char_v = one_hot(char2int[char], len(int2char)).unsqueeze(0)
        char_vectors.append(char_v)
        
    return th.cat(char_vectors,dim =0)

In [0]:
matrix = string2one_hot_matrix("Hello")


In [0]:
matrix.shape

torch.Size([8, 69])

In [0]:
char2index = char2int
index2char = int2char

In [0]:
one_hot(char2index['p'],len(index2char))

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [0]:
matrix

In [0]:
#keys are str_a,str_b
str_a = string2one_hot_matrix("abcdf")
str_b = string2one_hot_matrix("Hello")

In [0]:
str_a

In [0]:
str_b

In [0]:
str_a * str_b

In [0]:
(str_a * str_b).sum()

tensor(3)

In [0]:
#key matching along dim=0 how  many char are overlapping
(str_a * str_b).sum(dim=0)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [0]:
#key matching along dim=1 how  many char are overlapping in particular index
vect = (str_a * str_b).sum(dim=1)
vect

tensor([0, 0, 0, 0, 0, 1, 1, 1])

In [0]:
#when all positions in keys match completely
vect = (str_a * str_b).sum(dim=1)
x = vect[0]

for i in range(vect.shape[0] - 1):
    x= x*vect[i+1]
    
key_match = x
key_match

tensor(0)

In [0]:
def strings_equal(str_a, str_b):
    vect = (str_a * str_b).sum(dim=1)
    x = vect[0]

    for i in range(vect.shape[0] - 1):
        x= x*vect[i+1]

    str_match = x
    
    return str_match

In [0]:
#start building DB

keys = list()
values = list()

keys.append(string2one_hot_matrix("key1"))
values.append(string2values("value1"))

keys.append(string2one_hot_matrix("key2"))
values.append(string2values("value2"))

In [0]:
query_str= "key1"
query_matrix = string2one_hot_matrix(query_str)

In [46]:
query_matrix.shape

torch.Size([8, 69])

In [0]:
key_matches = list()
for key in keys:
    key_match = strings_equal(key, query_matrix)
    key_matches.append(key_match)

In [0]:
key_matches

[tensor(1), tensor(0)]

In [0]:
values

[tensor([22,  1, 12, 21,  5, 28, 50, 50]),
 tensor([22,  1, 12, 21,  5, 29, 50, 50])]

In [0]:
value_match = values[0] * key_matches[0]
value_match

tensor([22,  1, 12, 21,  5, 28, 50, 50])

In [0]:
values[1] * key_matches[1]

tensor([0, 0, 0, 0, 0, 0, 0, 0])

In [0]:
value_match = values[0] * key_matches[0]

for i in range(len(values) - 1):
    value_match += values[i+1]*key_matches[i+1]
    
value_match

tensor([22,  1, 12, 21,  5, 28, 50, 50])

In [0]:
result = values[0] * key_matches[0]

for i in range(len(values) - 1):
    result += values[i+1]*key_matches[i+1]
    
result

tensor([22,  1, 12, 21,  5, 28, 50, 50])

In [0]:
values2string(result).replace(".","")

'value1'

In [0]:
def query(query_str):
    query_matrix = string2one_hot_matrix(query_str)
    
    key_matches = list()
    
    for key in keys:
        key_match = strings_equal(key, query_matrix)
        key_matches.append(key_match)
    
    result = values[0] * key_matches[0]
    
    for i in range(len(values) - 1):
        result += values[i+1]*key_matches[i+1]
    
    return values2string(result).replace(".","")
    

In [0]:
query("key2")

'value2'

#final version of  database (not encrypted yet!)

In [0]:
def string2values (str_input, max_len = 8):
  
    str_input = str_input[:max_len].lower()
    
    #pad strings shorte than max len
    if (len(str_input)<max_len):
        str_input = str_input + "."*(max_len -len(str_input))
        
    values= list()
    for char in str_input :
        values.append(char2int[char])
        
    return th.tensor(values).long()
  
  
def values2string(input_value):
    
    s = ""
    
    for value in input_value:
      s += int2char[int(value)]
    return s
        
def strings_equal(str_a, str_b):
    vect = (str_a * str_b).sum(dim=1)
    x = vect[0]

    for i in range(vect.shape[0] - 1):
        x= x*vect[i+1]

    #str_match = x
    
    return x   #str_match
  
  
def one_hot(index,length):
    vect =  th.zeros(length).long()
    vect[index] = 1
    return vect
  
def string2one_hot_matrix(str_input,max_len = 8):
    
    str_input = str_input[:max_len].lower()
    
    #pad strings shorte than max len
    if (len(str_input)<max_len):
        str_input = str_input + "."*(max_len -len(str_input))
    
    char_vectors = list()
    for char in str_input:
        char_v = one_hot(char2int[char], len(int2char)).unsqueeze(0)
        char_vectors.append(char_v)
        
    return th.cat(char_vectors,dim =0)

In [0]:
class DB ():
    def __init__(self, max_key_len = 8, max_val_len = 8):
        self.max_key_len = 8
        self.max_val_len = 8

        self.keys = list()
        self.values = list()

        #keys.append(string2one_hot_matrix("key1"))
        #values.append(string2values("value1"))

        #keys.append(string2one_hot_matrix("key2"))
        #values.append(string2values("value2"))
      
    
    def add_entry(self, key, value):
        self.keys.append(string2one_hot_matrix(key))
        self.values.append(string2values(value))
    
    def query(self, query_str):
        query_matrix = string2one_hot_matrix(query_str)

        key_matches = list()

        for key in self.keys:
            key_match = strings_equal(key, query_matrix)
            key_matches.append(key_match)

        result = self.values[0] * key_matches[0]

        for i in range(len(self.values) - 1):
            result += self.values[i+1]*key_matches[i+1]

        return values2string(result).replace(".","")
    


In [0]:
db = DB()


In [0]:
db.add_entry("key1", "value1")
db.add_entry("key2", "value2")
db.add_entry("key3", "value3")
db.add_entry("key4", "value4")

In [23]:
db.query("key3")

'value3'

In [0]:
db = EncryptedDB()

db.add_entry("key1", "value1")
db.add_entry("key2", "value2")
db.add_entry("key3", "value3")
db.add_entry("key4", "value4")

db.query("key1")

'value1'

#now encrypted database

In [0]:
bob = sy.VirtualWorker(hook, id="bob")#.add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice")#.add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker")#.add_worker(sy.local_worker)
#me = sy.local_worker

In [0]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [25]:
alice._objects

{}

In [26]:
bob._objects

{}

In [27]:
secure_worker._objects

{}

In [0]:
sy.local_worker._objects

{}

In [0]:
xpt = x.share(alice,bob,secure_worker)

In [0]:
xpt

(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:30159209386 -> alice:95588463243]
	-> [PointerTensor | me:8776402290 -> bob:18229401244]
	-> [PointerTensor | me:62929947250 -> secure_worker:88904705586]
	*crypto provider: me*

In [0]:
xpt.get()

tensor([1, 2, 3, 4])

In [0]:
class EncryptedDB ():
    def __init__(self, *owners, max_key_len = 8, max_val_len = 8):
        self.max_key_len = max_key_len
        self.max_val_len = max_val_len

        self.keys = list()
        self.values = list()
        self.owners = owners

           
    def add_entry(self, key, value):
        key = string2one_hot_matrix(key)
        key = key.share(*self.owners)
        self.keys.append(key)
        
        value = string2values(value, max_len = self.max_val_len)
        value = value.share(*self.owners)
        self.values.append(value)
        
    
    def query(self, query_str):
      
        query_matrix = string2one_hot_matrix(query_str)
        
        query_matrix = query_matrix.share(*self.owners)

        key_matches = list()

        for key in self.keys:
            key_match = strings_equal(key, query_matrix)
            key_matches.append(key_match)

        result = self.values[0] * key_matches[0]

        for i in range(len(self.values) - 1):
            result += self.values[i+1]*key_matches[i+1]
            
        result = result.get()

        return values2string(result).replace(".","")
    


In [0]:
####################
db = EncryptedDB()

db.add_entry("key1", "value1")
db.add_entry("key2", "value2")
db.add_entry("key3", "value3")
db.add_entry("key4", "value4")

db.query("key1")

In [0]:
db = EncryptedDB(alice, bob, secure_worker)

In [30]:
db.owners

(<VirtualWorker id:alice #objects:0>,
 <VirtualWorker id:bob #objects:0>,
 <VirtualWorker id:secure_worker #objects:0>)

In [31]:
alice._objects

{}

In [32]:
bob._objects

{}

In [33]:
secure_worker._objects

{}

In [0]:
db.add_entry("key1", "value1")
db.add_entry("key2", "value2")
db.add_entry("key3", "value3")
db.add_entry("key4", "value4")


In [0]:
keys
4216191979
15318099606
37470906307
18597796201
values
6695358831
46719076640
51419330771
92914599600

query_matrix
17493031172

In [0]:
alice._objects

In [0]:
bob._objects

In [0]:
secure_worker._objects

In [39]:
db.keys

[(Wrapper)>[AdditiveSharingTensor]
 	-> [PointerTensor | me:90897264803 -> alice:4216191979]
 	-> [PointerTensor | me:53155810295 -> bob:48204390003]
 	-> [PointerTensor | me:38783002157 -> secure_worker:45107109400]
 	*crypto provider: me*, (Wrapper)>[AdditiveSharingTensor]
 	-> [PointerTensor | me:99927475188 -> alice:15318099606]
 	-> [PointerTensor | me:98225236947 -> bob:84849998016]
 	-> [PointerTensor | me:16658719745 -> secure_worker:86665269736]
 	*crypto provider: me*, (Wrapper)>[AdditiveSharingTensor]
 	-> [PointerTensor | me:34389056001 -> alice:37470906307]
 	-> [PointerTensor | me:70878039843 -> bob:48601300304]
 	-> [PointerTensor | me:47960201318 -> secure_worker:55207703700]
 	*crypto provider: me*, (Wrapper)>[AdditiveSharingTensor]
 	-> [PointerTensor | me:84714719192 -> alice:18597796201]
 	-> [PointerTensor | me:81360089714 -> bob:35425914299]
 	-> [PointerTensor | me:80443806140 -> secure_worker:17843737877]
 	*crypto provider: me*]

In [40]:
db.values

[(Wrapper)>[AdditiveSharingTensor]
 	-> [PointerTensor | me:39348845534 -> alice:46719076640]
 	-> [PointerTensor | me:31741260491 -> bob:96893646631]
 	-> [PointerTensor | me:12998955303 -> secure_worker:41972362799]
 	*crypto provider: me*, (Wrapper)>[AdditiveSharingTensor]
 	-> [PointerTensor | me:65877521520 -> alice:51419330771]
 	-> [PointerTensor | me:17525286157 -> bob:59216608605]
 	-> [PointerTensor | me:68378697911 -> secure_worker:74973745835]
 	*crypto provider: me*, (Wrapper)>[AdditiveSharingTensor]
 	-> [PointerTensor | me:64917502320 -> alice:92914599600]
 	-> [PointerTensor | me:14678965993 -> bob:38948640405]
 	-> [PointerTensor | me:77352592664 -> secure_worker:49120055480]
 	*crypto provider: me*, (Wrapper)>[AdditiveSharingTensor]
 	-> [PointerTensor | me:77675190296 -> alice:6695358831]
 	-> [PointerTensor | me:72210380934 -> bob:93948576586]
 	-> [PointerTensor | me:6651822182 -> secure_worker:94705742890]
 	*crypto provider: me*]

In [48]:
query_str = "key1"
query_matrix = string2one_hot_matrix(query_str)
        
query_matrix

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [49]:
query_matrix.shape

torch.Size([8, 69])

In [0]:
query_matrix = query_matrix.share(*db.owners)

In [52]:
query_matrix

(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:67092735802 -> alice:17493031172]
	-> [PointerTensor | me:99545348958 -> bob:58824104386]
	-> [PointerTensor | me:10764715448 -> secure_worker:55953103344]
	*crypto provider: me*

In [0]:
alice._objects

In [54]:
key_matches = list()

for key in db.keys:
    key_match = strings_equal(key, query_matrix)
    key_matches.append(key_match)

TypeError: ignored

In [56]:
db.keys[0]

(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:90897264803 -> alice:4216191979]
	-> [PointerTensor | me:53155810295 -> bob:48204390003]
	-> [PointerTensor | me:38783002157 -> secure_worker:45107109400]
	*crypto provider: me*

In [0]:
def strings_equal(str_a, str_b):
    vect = (str_a * str_b).sum(dim=1)
    x = vect[0]

    for i in range(vect.shape[0] - 1):
        x= x*vect[i+1]

    #str_match = x
    
    return x   #str_match
  

In [0]:
def strings_equal(str_a, str_b):

    vect = (str_a * str_b).sum(1)

    x = vect[0]

    for i in range(vect.shape[0] - 1):
        x = x * vect[i + 1]    

    return x

In [59]:
db.keys[0]*query_matrix

TypeError: ignored

In [60]:
db.query("key1")

TypeError: ignored

In [0]:
query_matrix.get()

In [0]:
db.keys[0]

(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:48350773864 -> alice:2951093093]
	-> [PointerTensor | me:91732805609 -> bob:70103198952]
	-> [PointerTensor | me:16313226171 -> secure_worker:10277804245]
	*crypto provider: me*

In [0]:
query_matrix *db.keys[0]

KeyError: ignored

In [0]:
query_matrix

In [0]:
query_matrix.get()

In [0]:
query_str

In [0]:
db = EncryptedDB(alice, bob, secure_worker)

db.add_entry("key1", "value1")
db.add_entry("key2", "value2")
db.add_entry("key3", "value3")
db.add_entry("key4", "value4")

db.query("key1")

In [0]:
db = encryptedDB(alice, bob, secure_worker, max_val_len = 256)

db.add_entry("Bob", "123 456 7890")
db.add_entry("Bill", "234 5678901")
db.add_entry("Sam", "345 6789012")
db.add_entry("key", "reallybignumber")

db.query("Bob")

# Lesson: Encrypted Deep Learning in PyTorch

### Train a Model

In [0]:
from torch import nn
from torch import optim
import torch.nn.functional as F

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# A Toy Model
model = Net()

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        
train()

tensor(0.9531)
tensor(1.3878)
tensor(5.8197)
tensor(13.6372)
tensor(7.8522)
tensor(0.9594)
tensor(0.7709)
tensor(0.7118)
tensor(0.6515)
tensor(0.5745)
tensor(0.4967)
tensor(0.4161)
tensor(0.3239)
tensor(0.2526)
tensor(0.1886)
tensor(0.1267)
tensor(0.0867)
tensor(0.0532)
tensor(0.0370)
tensor(0.0223)


In [0]:
model(data)

tensor([[ 0.0514],
        [-0.0101],
        [ 0.9566],
        [ 0.8994]], grad_fn=<AddmmBackward>)

## Encrypt the Model and Data

In [0]:
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [0]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:90270312706 -> alice:64621591736]
 	-> (Wrapper)>[PointerTensor | me:26346950071 -> bob:21579874363]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:5156179829 -> alice:87001562796]
 	-> (Wrapper)>[PointerTensor | me:16697713904 -> bob:74982329939]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:96063569101 -> alice:60924746524]
 	-> (Wrapper)>[PointerTensor | me:7997403527 -> bob:91730833656]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:36221792128 -> alice:18267234054]
 	-> (Wrapper)>[PointerTensor | me:61423632792 -> bob:8162820659

In [0]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [0]:
encrypted_data

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:48419950975 -> alice:61588180745]
	-> (Wrapper)>[PointerTensor | me:24805025079 -> bob:2592329358]
	*crypto provider: secure_worker*

In [0]:
encrypted_prediction = encrypted_model(encrypted_data)

In [0]:
encrypted_prediction.get().float_precision()

tensor([[ 0.0520],
        [-0.0100],
        [ 0.9560],
        [ 0.8990]])

# Lesson: Encrypted Deep Learning in Keras


## Step 1: Public Training

Welcome to this tutorial! In the following notebooks you will learn how to provide private predictions. By private predictions, we mean that the data is constantly encrypted throughout the entire process. At no point is the user sharing raw data, only encrypted (that is, secret shared) data. In order to provide these private predictions, Syft Keras uses a library called [TF Encrypted](https://github.com/tf-encrypted/tf-encrypted) under the hood. TF Encrypted combines cutting-edge cryptographic and machine learning techniques, but you don't have to worry about this and can focus on your machine learning application.

You can start serving private predictions with only three steps:
- **Step 1**: train your model with normal Keras.
- **Step 2**: secure and serve your machine learning model (server).
- **Step 3**: query the secured model to receive private predictions (client). 

Alright, let's go through these three steps so you can deploy impactful machine learning services without sacrificing user privacy or model security.

Huge shoutout to the Dropout Labs ([@dropoutlabs](https://twitter.com/dropoutlabs)) and TF Encrypted ([@tf_encrypted](https://twitter.com/tf_encrypted)) teams for their great work which makes this demo possible, especially: Jason Mancuso ([@jvmancuso](https://twitter.com/jvmancuso)), Yann Dupis ([@YannDupis](https://twitter.com/YannDupis)), and Morten Dahl ([@mortendahlcs](https://github.com/mortendahlcs)). 

_Demo Ref: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials_

## Train Your Model in Keras

To use privacy-preserving machine learning techniques for your projects you should not have to learn a new machine learning framework. If you have basic [Keras](https://keras.io/) knowledge, you can start using these techniques with Syft Keras. If you have never used Keras before, you can learn a bit more about it through the [Keras documentation](https://keras.io). 

Before serving private predictions, the first step is to train your model with normal Keras. As an example, we will train a model to classify handwritten digits. To train this model we will use the canonical [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

We borrow [this example](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py) from the reference Keras repository.  To train your classification model, you just run the cell below.

In [0]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Activation

batch_size = 128
num_classes = 10
epochs = 2

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


Train on 60000 samples, validate on 10000 samples
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Epoch 1/2
60000/60000 [==============================] - 7s 119us/sample - loss: 0.6739 - acc: 0.7855 - val_loss: 0.2742 - val_acc: 0.9135
Epoch 2/2
60000/60000 [==============================] - 7s 122us/sample - loss: 0.2181 - acc: 0.9341 - val_loss: 0.1549 - val_acc: 0.9517
Test loss: 0.1548735132828355
Test accuracy: 0.9517


In [0]:
## Save your model's weights for future private prediction
model.save('short-conv-mnist.h5')

## Step 2: Load and Serve the Model

Now that you have a trained model with normal Keras, you are ready to serve some private predictions. We can do that using Syft Keras.

To secure and serve this model, we will need three TFEWorkers (servers). This is because TF Encrypted under the hood uses an encryption technique called [multi-party computation (MPC)](https://en.wikipedia.org/wiki/Secure_multi-party_computation). The idea is to split the model weights and input data into shares, then send a share of each value to the different servers. The key property is that if you look at the share on one server, it reveals nothing about the original value (input data or model weights).

We'll define a Syft Keras model like we did in the previous notebook. However, there is a trick: before instantiating this model, we'll run `hook = sy.KerasHook(tf.keras)`. This will add three important new methods to the Keras Sequential class:
 - `share`: will secure your model via secret sharing; by default, it will use the SecureNN protocol from TF Encrypted to secret share your model between each of the three TFEWorkers. Most importantly, this will add the capability of providing predictions on encrypted data.
 - `serve`: this function will launch a serving queue, so that the TFEWorkers can can accept prediction requests on the secured model from external clients.
 - `shutdown_workers`: once you are done providing private predictions, you can shut down your model by running this function. It will direct you to shutdown the server processes manually if you've opted to manually manage each worker.

If you want learn more about MPC, you can read this excellent [blog](https://mortendahl.github.io/2017/04/17/private-deep-learning-with-mpc/).

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

import syft as sy
hook = sy.KerasHook(tf.keras)

## Model

As you can see, we define almost the exact same model as before, except we provide a `batch_input_shape`. This allows TF Encrypted to better optimize the secure computations via predefined tensor shapes. For this MNIST demo, we'll send input data with the shape of (1, 28, 28, 1). 
We also return the logit instead of softmax because this operation is complex to perform using MPC, and we don't need it to serve prediction requests.

In [0]:
num_classes = 10
input_shape = (1, 28, 28, 1)

model = Sequential()

model.add(Conv2D(10, (3, 3), batch_input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, name="logit"))

### Load Pre-trained Weights

With `load_weights` you can easily load the weights you have saved previously after training your model.

In [0]:
pre_trained_weights = 'short-conv-mnist.h5'
model.load_weights(pre_trained_weights)

## Step 3: Setup Your Worker Connectors

Let's now connect to the TFEWorkers (`alice`, `bob`, and `carol`) required by TF Encrypted to perform private predictions. For each TFEWorker, you just have to specify a host.

These workers run a [TensorFlow server](https://www.tensorflow.org/api_docs/python/tf/distribute/Server), which you can either manage manually (`AUTO = False`) or ask the workers to manage for you (`AUTO = True`). If choosing to manually manage them, you will be instructed to execute a terminal command on each worker's host device after calling `model.share()` below.  If all workers are hosted on a single device (e.g. `localhost`), you can choose to have Syft automatically manage the worker's TensorFlow server.

In [0]:
AUTO = False

alice = sy.TFEWorker(host='localhost:4000', auto_managed=AUTO)
bob = sy.TFEWorker(host='localhost:4001', auto_managed=AUTO)
carol = sy.TFEWorker(host='localhost:4002', auto_managed=AUTO)

## Step 4: Split the Model Into Shares

Thanks to `sy.KerasHook(tf.keras)` you can call the `share` method to transform your model into a TF Encrypted Keras model.

If you have asked to manually manage servers above then this step will not complete until they have all been launched. Note that your firewall may ask for Python to accept incoming connection.

In [0]:
model.share(alice, bob, carol)

INFO:tf_encrypted:If not done already, please launch the following command in a terminal on host 'localhost:4000':
'python -m tf_encrypted.player --config /tmp/tfe.config server0'
This can be done automatically in a local subprocess by setting `auto_managed=True` when instantiating a TFEWorker.
INFO:tf_encrypted:If not done already, please launch the following command in a terminal on host 'localhost:4001':
'python -m tf_encrypted.player --config /tmp/tfe.config server1'
This can be done automatically in a local subprocess by setting `auto_managed=True` when instantiating a TFEWorker.
INFO:tf_encrypted:If not done already, please launch the following command in a terminal on host 'localhost:4002':
'python -m tf_encrypted.player --config /tmp/tfe.config server2'
This can be done automatically in a local subprocess by setting `auto_managed=True` when instantiating a TFEWorker.
INFO:tf_encrypted:Starting session on target 'grpc://localhost:4000' using config graph_options {
}



## Step 5: Launch 3 Servers

```
python -m tf_encrypted.player --config /tmp/tfe.config server0
python -m tf_encrypted.player --config /tmp/tfe.config server1
python -m tf_encrypted.player --config /tmp/tfe.config server2```

## Step 6: Serve the Model

Perfect! Now by calling `model.serve`, your model is ready to provide some private predictions. You can set `num_requests` to set a limit on the number of predictions requests served by the model; if not specified then the model will be served until interrupted.

In [0]:
model.serve(num_requests=3)

Served encrypted prediction 1 to client.
Served encrypted prediction 2 to client.
Served encrypted prediction 3 to client.


## Step 7: Run the Client

At this point open up and run the companion notebook: Section 4b - Encrytped Keras Client

## Step 8: Shutdown the Servers

Once your request limit above, the model will no longer be available for serving requests, but it's still secret shared between the three workers above. You can kill the workers by executing the cell below.

**Congratulations** on finishing Part 12: Secure Classification with Syft Keras and TFE!

In [0]:
model.shutdown_workers()

if not AUTO:
    process_ids = !ps aux | grep '[p]ython -m tf_encrypted.player --config /tmp/tfe.config' | awk '{print $2}'
    for process_id in process_ids:
        !kill {process_id}
        print("Process ID {id} has been killed.".format(id=process_id))

# Keystone Project - Mix and Match What You've Learned

Description: Take two of the concepts you've learned about in this course (Encrypted Computation, Federated Learning, Differential Privacy) and combine them for a use case of your own design. Extra credit if you can get your demo working with [WebSocketWorkers](https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials/advanced/websockets-example-MNIST) instead of VirtualWorkers! Then take your demo or example application, write a blogpost, and share that blogpost in #general-discussion on OpenMined's slack!!!

Inspiration:
- This Course's Code: https://github.com/Udacity/private-ai
- OpenMined's Tutorials: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials
- OpenMined's Blog: https://blog.openmined.org